In [10]:
from oodtk import Mahalanobis, Softmax
from oodtk.utils import is_unknown
from oodtk.model import WideResNet
from oodtk.transforms import ToRGB

model = WideResNet.from_pretrained("cifar10-pt", num_classes=10)

In [11]:
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import torchvision.transforms as tvt
import torch

torch.manual_seed(123)

mean = [x / 255 for x in [125.3, 123.0, 113.9]]
std = [x / 255 for x in [63.0, 62.1, 66.7]]

trans = tvt.Compose([ToRGB(), tvt.Resize((32,32)), tvt.ToTensor(), tvt.Normalize(mean, std)])

data_train = CIFAR10(root="data", train=True, download=True, transform=trans)
data_test = CIFAR10(root="data", train=False, download=True, transform=trans)


test_loader = DataLoader(data_test, batch_size=128)

Files already downloaded and verified
Files already downloaded and verified


In [12]:
from torchmetrics import AUROC, Accuracy
from tqdm.notebook import tqdm

# softmax = Softmax(model).cuda()
# auroc_softmax = AUROC(num_classes=2)

acc = Accuracy().cuda()

model.eval()
model.cuda()

with torch.no_grad():
    for batch in tqdm(test_loader):
        x, y = batch
        x = x.cuda()
        y = y.cuda()

        y_hat = model(x)
        acc.update(y_hat, y)
        # auroc_softmax.update(softmax(x), is_unknown(y))
        break

print(acc.compute())

  0%|          | 0/79 [00:00<?, ?it/s]

tensor(0.9688, device='cuda:0')


In [16]:
train_loder = DataLoader(data_train, shuffle=True)
d = train_loder
maha = Mahalanobis(model).cuda()
maha.fit(d)

KeyboardInterrupt: 